In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.nn import Conv2d
from torchvision.transforms import Compose
from torchvision.transforms import Resize
from torchvision.transforms import CenterCrop
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor
import numpy as np

In [ ]:
import os
from torch.utils.data import random_split

In [ ]:
data = '/content/drive/MyDrive/TrainingData_Final'
print()
dirs = os.listdir(data)
dataset_old = []
for dir in dirs:
  for inner_dir in os.listdir(os.path.join(data, dir)):
      dataset_old.append(os.path.join(data, dir, inner_dir))
dataset = []
curr = 0
for i in dataset_old:
  if i[42] != '5':
    dataset.append(i)
  elif curr < 400:
    dataset.append(i)
    curr += 1

In [ ]:
train_set = []
test_set = []
validation_set = []
for i in range(0, 6):
  path = f"/content/drive/MyDrive/TrainingData_Final/{i}"
  all_imgs_0 = os.listdir(path)
  all_imgs_0 = [f"{path}/{_path}" for _path in all_imgs_0]
  if i == 5:
    all_imgs_0 = all_imgs_0[:400]
  num_imgs_0 = len(all_imgs_0)
  arr = np.arange(num_imgs_0)
  np.random.shuffle(arr)
  curr_tr = len(train_set)
  curr_te = len(test_set)
  curr_va = len(validation_set)
  train_set += [all_imgs_0[idx] for idx in arr[0:int(0.8*num_imgs_0)]]
  test_set += [all_imgs_0[idx]  for idx in arr[int(0.8*num_imgs_0):int(0.9*num_imgs_0)]]
  validation_set += [all_imgs_0[idx]  for idx in arr[int(0.9*num_imgs_0):int(num_imgs_0)]]
  print(i, 'train: ', len(train_set) - curr_tr)
  print(i, 'test: ', len(test_set) - curr_te)
  print(i, 'val: ', len(validation_set) - curr_va)
  print('---')
print('total:', len(train_set) + len(test_set) + len(validation_set))
print('train:', len(train_set))
print('test:', len(test_set))
print('val:', len(validation_set))

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

labels = []
class CustomDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        image = Image.open(file_path)

        if self.transform:
            image = self.transform(image)


        return image, file_path[42]
train_dataloader = DataLoader(CustomDataset(train_set), batch_size=64, shuffle=True)
test_dataloader = DataLoader(CustomDataset(test_set), batch_size=64, shuffle=True)
val_dataloader = DataLoader(CustomDataset(validation_set), batch_size=64, shuffle=True)

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [ ]:
train_features, train_labels = next(iter(train_dataloader))
img = train_features[0].squeeze()
label = train_labels[0]

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.linear_relu_stack = nn.Sequential(
      nn.Conv2d(3, 4, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(4, 8, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(8, 16, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Flatten(),
      nn.Linear(14400, 5000),
      nn.Linear(5000, 5),
      nn.Softmax(dim=1)
    )
  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
class NeuralNetwork2(nn.Module):
  def __init__(self):
    super(NeuralNetwork2, self).__init__()
    self.linear_relu_stack = nn.Sequential(
      nn.Conv2d(3, 4, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(4, 8, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(8, 16, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Flatten(),
      nn.Linear(14400, 5000),
      nn.Linear(5000, 2),
    )
  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
# 5-class
def label_changer(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 3 or y[i] == 5:
      y[i] = 0
    elif y[i] != 4:
      y[i] += 1
    num = y[i]
    y[i] = torch.tensor([[0, 0, 0, 0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  return y
from re import I
model = NeuralNetwork().to(device)
print(model)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    optimizer.zero_grad()
    X = X.cpu()
    y = list(y)
    y = label_changer(y)
    y = y.cpu()
    pred = model(X)
    pred = pred.to(torch.float32)
    y = y.to(torch.float32)
    loss = loss_fn(pred, y.argmax(1))
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    accuracy = (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    print(accuracy)
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for batch, (X, y) in enumerate(dataloader):
      X = X.cpu()
      y = list(y)
      y = label_changer(y)
      y = y.cpu()
      pred = model(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 20
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
# label 0: Mild Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 0:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
      print(pred.argmax(1))
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary0model")

In [ ]:
# label 1: Moderate Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 1:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary1model")

In [ ]:
# label 2: Severe Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 2:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary2model")

In [ ]:
# label 3: Hypertensive  Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 3:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary3model")

In [ ]:
# label 4: Severe Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 4:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary4model")